In [3]:
import pandas as pd
import numpy as np
import gensim

In [4]:
path = '/Users/deveshkandpal/Code/ADS/final-project/dataset/dataset/Step6/sentiment-nc/nc_sentiment.json'
nc_df = pd.read_json(path, orient='records', lines=True)
nc_df.head()

,address,name,review_review_id,review_stars,review_text,stars,unique_business_id
0,4801-P N Tryon St,Taqueria Tóluca,2kbvCQfmxfPSYGCnsl5tPw,3,"If you don't speak Spanish, it is not a proble...",3.0,21516
1,2121 Shamrock Dr,Letty's,B8MWqpia_A2SvEPCdhIBWg,5,Love Letty's! it is off the beaten path (for m...,4.0,21262
2,4801-P N Tryon St,Taqueria Tóluca,5WlLB5jxCJ-5M8N4ID8GoA,2,"The food wasn't great, and our waitress wasn't...",3.0,21516
3,2121 Shamrock Dr,Letty's,rvM9x7GIFc84xXM8SsdQjw,4,"I was pretty sad to see Foskoskie's close, but...",4.0,21262
4,4801-P N Tryon St,Taqueria Tóluca,qUSAl2ATzTBeTI1DOkSvIA,5,Awesome prices for the amount of food!!! Very ...,3.0,21516


In [30]:
from textblob import TextBlob
import re
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem.porter import PorterStemmer as ps


In [39]:
rest_review_dict = dict()
for temp in nc_df.iterrows() :
    row = temp[1]
    business_id = row.unique_business_id
    
    review_text = row['review_text']
    text = ' '.join([word for word in review_text.lower().split() if word not in STOPWORDS])
    
    review_stars = row['stars']
    if business_id in rest_review_dict :
        reviews_array = rest_review_dict[business_id]
        reviews_array.append({'review_text' : review_text, 'review_stars' : review_stars,
                              'polarity' : TextBlob(text).sentiment.polarity,
                             'stemmed_text' : text})
    else :
        reviews_array = list()
        reviews_array.append({'review_text' : review_text, 'review_stars' : review_stars,
                              'polarity' : TextBlob(text).sentiment.polarity,
                             'stemmed_text' : text})
        rest_review_dict[business_id] = reviews_array
    
        
        
    

In [47]:
arr = rest_review_dict.get(21516)
from gensim import corpora
from gensim import corpora, models, similarities

In [111]:
review_array =  rest_review_dict.get(21516)
consolidated_text = [review['stemmed_text'] for review in review_array]
texts = []
for t in consolidated_text :
    for word in t.split(" ") :
        texts.append(word)
        
texts = [texts]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2)
topics = dict()
for topic in lda.top_topics(corpus) :
    
    b = topic[0][0:15]
    for tup in b :
        if tup[1] not in topics :
            topics[tup[1]] = tup[0]
        else :
            if topics[tup[1]] < tup[0] :
                topics[tup[1]] = tup[0]

print(topics)

{'food': 0.019300087922043567, 'it.': 0.013293171822670485, 'spanish': 0.012694335108246562, "don't": 0.01217728649715393, 'lunch': 0.01329392054010238, "wasn't": 0.010930028305032874, 'flavor': 0.01016610339847515, 'problem': 0.0096716426354236402, 'little': 0.0096290855596992131, 'wanted': 0.0095589579875599521, 'quick': 0.0093149456014753294, 'waitress': 0.0091469668903295751, 'lot': 0.0091383133684499653, 'ok.': 0.009126060572372181, 'beans': 0.0094422368138282666, 'order': 0.010273908977596936, 'time': 0.01026160635015946, 'chicken': 0.010031945459910922, 'speak': 0.0095839124656104963, 'mexican': 0.0094576763105320132}


[([(0.063249853977684484, 'ok.'),
   (0.049883759673421846, 'chicken'),
   (0.032190698970755403, "don't"),
   (0.031978614688191277, 'told'),
   (0.031686174376962431, 'salsa'),
   (0.031074942823033126, 'salsas'),
   (0.031010197802306905, 'restaurant.com'),
   (0.030822468180346705, 'tried'),
   (0.030727118085907505, 'ordered'),
   (0.030593735421666442, 'rice'),
   (0.030585872340587426, 'smothered'),
   (0.030285545361259943, 'salsa.'),
   (0.029970374677443574, 'fraud.'),
   (0.029469669485816079, 'server'),
   (0.02926729950066401, 'warm'),
   (0.029230013389030397, 'spicy'),
   (0.029155663860851246, 'lunch'),
   (0.029110052197997383, 'different'),
   (0.028835183036017835, 'printed'),
   (0.028534880701156149, "weren't")],
  -0.069314718051007695),
 ([(0.066568940166226914, 'chicken'),
   (0.04394267733644238, 'ok.'),
   (0.034934016802342947, 'accept'),
   (0.034406648271050895, 'chunky'),
   (0.034035362999831134, 'beans'),
   (0.03327482179929022, 'specials'),
   (0.03316

In [112]:
business_corpus = dict()
for business_id, review_array in rest_review_dict.items():
    consolidated_text = [review['stemmed_text'] for review in review_array]
    texts = []
    for t in consolidated_text :
        for word in t.split(" ") :
            texts.append(word)
    texts = [texts]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2)
    topics = dict()
    for topic in lda.top_topics(corpus) :
        b = topic[0][0:15]
        for tup in b :
            if tup[1] not in topics :
                topics[tup[1]] = tup[0]
            else :
                if topics[tup[1]] < tup[0] :
                    topics[tup[1]] = tup[0]
    
    
    business_corpus[business_id] = topics

        
    


In [114]:
tc = business_corpus.get(21516)
tc

{'beans': 0.009336599443797668,
 'chicken': 0.0097704353965480713,
 "don't": 0.012240668735138441,
 'flavor': 0.0095622695384658778,
 'food': 0.01926921365985005,
 'it.': 0.01355023601693553,
 'little': 0.00957380604109603,
 'lot': 0.0093019287698584679,
 'lunch': 0.013252030797275756,
 'mexican': 0.0091612458732610123,
 'ok.': 0.0091553878306199037,
 'order': 0.0096577009850250008,
 'problem': 0.0096345025747524883,
 'quick': 0.009073993903368209,
 'spanish': 0.013642005068855776,
 'speak': 0.0098450643199932599,
 'time': 0.009741084487520266,
 'waitress': 0.0094395045732568392,
 'wanted': 0.0091717859368537491,
 "wasn't": 0.0096380497108035559}

In [119]:
import pickle
pickle_out = open("nc_sentiment_corpus.json","wb")
pickle.dump(business_corpus, pickle_out)
pickle_out.close()

In [122]:
test_pkl = pickle.load( open( "nc_sentiment_corpus.json", "rb" ) )

In [126]:

pickle_out = open("nc_bus_review_dict.json","wb")
pickle.dump(rest_review_dict, pickle_out)
pickle_out.close()

In [5]:
test_pkl = pickle.load( open( "/Users/deveshkandpal/Code/ADS/Yelp_Data_Analysis/sentiment-models/NC/nc_bus_review_dict.json", "rb" ) )
len(test_pkl.keys())

3780

In [6]:
import pickle
test_pkl = pickle.load( open( "/Users/deveshkandpal/Code/ADS/Yelp_Data_Analysis/sentiment-models/NC/nc_sentiment_corpus.json", "rb" ) )
len(test_pkl.keys())

3780